<a href="https://colab.research.google.com/github/bartoszpankratz/VAR_neural_network/blob/master/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os.path import join
import pandas as pd
import scipy.stats as sps
import numpy as np

from statsmodels.tsa.arima_process import ArmaProcess
from arch import arch_model
from arch.univariate import EGARCH
from arch.univariate import HARCH

class Data:
    def __init__(self, path = None, file = None, params = None):
        if params is None:
            if file.find('.csv') == -1:
                self.ticker = file
                self.name   = file
            else:
                self.ticker = file[0:file.find('.csv')]
            self.path = join(path, file)
        else:
            self.ticker = params['PROCESS']
            self.name   = params['PROCESS']
            self.path   = None
            self.sim_data(params)
        
    def get_data(self, dictionary):
        dict_data      = dictionary[dictionary['Ticker'] == self.ticker]
        if len(dict_data) != 1:
            print(len(dict_data))
            return None
        else:
            data          = pd.read_csv(self.path)
            self.name     = dict_data['Name'].values[0]
            self.data = pd.DataFrame(data = {
                'Date'    : data['Data'][1:],
                'Prices'  : data['Zamkniecie'][1:],
                'Returns' : data['Zamkniecie'][1:].values 
                            / data['Zamkniecie'][:-1].values - 1
                })
            self.data = self.data.set_index('Date')

    def sim_data(self, SIM_PARAM):
        # Gaussian Noise
        if SIM_PARAM['PROCESS'] == 'GWS':
            #SIM_PARAM = {'PROCESS': 'GWS', 
            #         'SIM_LEN': 10000}
            sim_len = SIM_PARAM['SIM_LEN']
            y = np.random.normal(0, 1, size= sim_len)
            true_var = np.repeat([sps.norm.ppf(0.01)], sim_len)

        # AR
        elif SIM_PARAM['PROCESS'] == 'AR':          
            #SIM_PARAM = {'PROCESS': 'AR', 
            #             'SIM_LEN': 200,
            #             'AR': [-0.999]}

            ar = np.array([1] + SIM_PARAM['AR'])
            ar_object = ArmaProcess(ar, [1])
            y = ar_object.generate_sample(nsample = SIM_PARAM['SIM_LEN'])

            true_var = sum([y[i:-(len(ar)-i)] * (-1*ar[len(ar)-i]) for i in range(1, len(ar))]) +  sps.norm.ppf(0.01)
            y = y[len(ar): ]

        # ARCH
        elif SIM_PARAM['PROCESS'] == 'ARCH':
            #SIM_PARAM = {'PROCESS': 'ARCH', 
            #         'SIM_LEN': 1000,
            #         'INIT': [0, 0.1],
            #         'P_PARAMS': [0.2, 0.1]}
            model = arch_model(None, p = len(SIM_PARAM['P_PARAMS']), o = 0, q = 0)
            param_vect = SIM_PARAM['INIT'] + SIM_PARAM['P_PARAMS'] 

            y = model.simulate(nobs= SIM_PARAM['SIM_LEN'], burn= 100, params = param_vect)

            true_var  = y['volatility'] * sps.norm.ppf(0.01)
            y = y['data'] 

        # GARCH
        elif SIM_PARAM['PROCESS'] == 'GARCH':
            #SIM_PARAM = {'PROCESS': 'GARCH', 
            #         'SIM_LEN': 1000,
            #         'INIT': [0, 0.1],
            #         'P_PARAMS': [0.2, 0.1], 
            #         'Q_PARAMS': [0.2, 0.1]}
            model = arch_model(None, p = len(SIM_PARAM['P_PARAMS']), o = 0, q = len(SIM_PARAM['Q_PARAMS']))
            param_vect = SIM_PARAM['INIT'] + SIM_PARAM['P_PARAMS'] + SIM_PARAM['Q_PARAMS']
            y = model.simulate(nobs= SIM_PARAM['SIM_LEN'], burn= 100, params = param_vect)
            true_var  = y['volatility'] * sps.norm.ppf(0.01)
            y = y['data']     

        # GJR GARCH
        elif SIM_PARAM['PROCESS'] == 'GJR_GARCH':
            #SIM_PARAM = {'PROCESS': 'GJR_GARCH', 
            #         'SIM_LEN': 1000,
            #         'INIT': [0, 0.1],
            #         'P_PARAMS': [0.2, 0.1],
            #         'O_PARAMS': [0.5],
            #         'Q_PARAMS': [0.2, 0.1]}
            model = arch_model(None, p = len(SIM_PARAM['P_PARAMS']),
                               o = len(SIM_PARAM['O_PARAMS']),
                               q = len(SIM_PARAM['Q_PARAMS']))
            param_vect = SIM_PARAM['INIT'] + SIM_PARAM['P_PARAMS'] + SIM_PARAM['O_PARAMS'] + SIM_PARAM['Q_PARAMS']
            y = model.simulate(nobs= SIM_PARAM['SIM_LEN'], burn= 100, params = param_vect)
            true_var  = y['volatility'] * sps.norm.ppf(0.01)
            y = y['data']

        # EGARCH
        elif SIM_PARAM['PROCESS'] == 'EGARCH':
            #SIM_PARAM = {'PROCESS': 'EGARCH', 
            #         'SIM_LEN': 1000,
            #         'INIT': [0, 0.1],
            #         'P_PARAMS': [0.4, 0.1],
            #         'Q_PARAMS': [0.7, 0.1]}
            model = EGARCH(p = len(SIM_PARAM['P_PARAMS']),
                               q = len(SIM_PARAM['Q_PARAMS']))
            param_vect = SIM_PARAM['INIT'] + SIM_PARAM['P_PARAMS'] + SIM_PARAM['Q_PARAMS']

            y = model.simulate(nobs= SIM_PARAM['SIM_LEN'], burn= 1000, parameters = param_vect, rng=np.random.standard_normal)

            true_var  = y[1] * sps.norm.ppf(0.01)
            y = y[0]

        # HARCH
        elif SIM_PARAM['PROCESS'] == 'HARCH':
            ################################################
            ######### REALIZACJI Z TRUE VAR ################

            #SIM_PARAM = {'PROCESS': 'HARCH', 
            #         'SIM_LEN': 1000,
            #         'LAGS': [1, 5, 17],
            #         'INIT': [1], 
            #         'PARAMS': [0.2, 0.05, 0.01]}
            model = HARCH(lags = SIM_PARAM['LAGS'])
            param_vect = SIM_PARAM['INIT'] + SIM_PARAM['PARAMS']
            y = model.simulate(nobs= SIM_PARAM['SIM_LEN'], burn= 1000, parameters = param_vect, rng=np.random.standard_normal)
            true_var  = y[1] * sps.norm.ppf(0.01)
            y = y[0]

        self.name = SIM_PARAM['PROCESS']
        self.data = pd.DataFrame(data = {
        'Returns' : y, 
        'True_VAR_0.01' : true_var
        })
        
        
        
        
        
        
        